## M345SC Lab 2

The course bitbucket repo has been updated a few times in the last week. Ensure that you can sync your online fork of the repo. If/when using git on MLC machines, you should first launch *tortoise git* from the software hub, and then you will be provided with git options when right-clicking on a file or folder

#### Task 1: Timing code

There are a few different approaches for timing code in Python.
You can use the *time* function in the time module as below:

In [1]:
import numpy as np
import time
X = list(np.random.randint(1,1000,800)) #uncomment and use if/as needed
t1 = time.time()
y = np.sin(np.outer(X,X))
t2 = time.time()
print("dt = ",t2-t1)

dt =  0.024047136306762695


And you can also use the timeit function:

In [2]:
%timeit np.sin(np.outer(X,X))

19.6 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


You will now investigate how the runing time for sorting algorithms depends on the list length. Functions for merge and insertion sorts are below.

In [3]:
def isort(X):
    """Sort X using insertion sort algorithm and return sorted array
    """

    S = X.copy()

    for i,x in enumerate(X[1:],1):
        #place x appropriately in partially sorted array, S
        for j in range(i-1,-1,-1):
            if S[j+1]<S[j]:
                S[j],S[j+1] = S[j+1],S[j]
            else:
                break
    return S

def merge(L,R):
    """Merge 2 sorted lists provided as input
    into a single sorted list
    """
    M = [] #Merged list, initially empty
    indL,indR = 0,0 #start indices
    nL,nR = len(L),len(R)

    #Add one element to M per iteration until an entire sublist
    #has been added
    for i in range(nL+nR):
        if L[indL]<R[indR]:
            M.append(L[indL])
            indL = indL + 1
            if indL>=nL:
                M.extend(R[indR:])
                break
        else:
            M.append(R[indR])
            indR = indR + 1
            if indR>=nR:
                M.extend(L[indL:])
                break
    return M

def mergesort(X):
    """Given a unsorted list, sort list using
    merge sort algorithm and return sorted list
    """

    n = len(X)

    if n==1:
        return X
    else:
        L = mergesort(X[:n//2])
        R = mergesort(X[n//2:])
        return merge(L,R)

1) How much does the run-time for insertion sort increase when the length of the initial, unsorted list is doubled? What do you expect?

In [4]:
#Add code below
#Run isort with N=2400
N=2400
X = list(np.random.randint(1,2*N,N))
t1 = time.time()
S = isort(X)
t2 = time.time()
dt1 = t2-t1
print("N=%d, dt=%f"%(N,dt1))
X2 = list(np.random.randint(1,4*N,2*N))
t1 = time.time()
S = isort(X2)
t2 = time.time()
dt2 = t2-t1
print("N=%d, dt=%f"%(N,dt2))
print("dt2/dt1=%f"%(dt2/dt1))

N=2400, dt=0.610451
N=2400, dt=2.452306
dt2/dt1=4.017205


Since insertion sort requires $O(N^2)$ time, we expect the run time to increase by a factor of 4 when $N$ is doubled. 

2) Now carry out the analagous analysis for mergesort.

In [34]:
#Run merge sort with N=100000, N = 100000^1.1
N=100000
X = list(np.random.randint(1,2*N,N))
t1 = time.time()
S = mergesort(X)
t2 = time.time()
dt1m = t2-t1
print("N=%d, dt=%f"%(N,dt1m))
X2 = list(np.random.randint(1,int(N**1.2),int(N**1.1)))
t1 = time.time()
S = mergesort(X2)
t2 = time.time()
dt2m = t2-t1
print("N=%d, dt=%f"%(int(N**1.1),dt2m))
print("dt2/dt1=%f"%(dt2m/dt1m))


N=100000, dt=0.679421
N=316227, dt=2.418382
dt2/dt1=3.559474


Since the runtime for mergesort (for large N) is $O(N log_2 N)$, we should consider the ratio, $r = N^{1.1} log_2(N^{1.1})/(N log_2(N))$. This simplifies to $1.1 N^{0.1} \approx 3.5$. Note that timing results can vary quite a bit from one run to the next which is why *timeit* averages over several runs.

3) Consider one *level* of mergesort where *m* sorted lists of length *p* are merged into *m/2* lists of length *2p* (assume that *m* is even), and we have $N = mp$. In lecture, the running time for one level was stated to be $O(N)$. This indicates the number of required operations is: $cN + f(N)$ with $lim_{N\rightarrow \infty} f/N = 0$ and $c$ a constant independent of N. Examine the code above and construct an (useful) upper bound for $c$. Does this bound depend on *m* or *p*?

**Ans:** In the worst-case scenario, when merging 2 length-p lists, the merge function requires 1 update of *i*, an update of either indL or indR, two comparisons, and one append per iteration with *2p* iterations. Additionally, before the loop, we have three assignments and two length calculations. So, we have *10p+5* operations per merge. There are *m/2* merges per level, so we have *5mp+5m/2* = *5N + 5m/2* operations per level. We know that $m \le N$ so we can say the running time for one level (for our implementation) is $T < 8N$ and *c=8*.   

####  Task 2: Setting up a word count
In this task, you will place the words from a text file into a Python dictionary so that word counts for specified words can be quickly extracted. The line below will read in the file (if the notebook and the text file, *words.txt*, are in the same directory).

In [6]:
infile = open('words.txt','r')
A = infile.read()
#Remove punctuation
A = A.replace(',','')
A = A.replace('?','')
A = A.replace('.','')
A = A.replace('!','')
A = A.lower() #Convert to all lowercase
words = A.split() #List (mostly) containing the words in the file
print(words[:4])
infile.close()

['sing', 'michael', 'sing', 'on']


1) Fill the dictionary below so that each word in *words* is stored as a key and the corresponding value is the number of times the word appears in the list. If you are unfamiliar with dictionaries in Python, have a look at the online tutorial: https://docs.python.org/3/tutorial/datastructures.html#dictionaries

In [7]:
D = {} #Initializes empty dictionary
#Add code below
for w in words:
    if w in D:
        D[w] = D[w]+1
    else:
        D[w]=1

2) Using your dictionary, look up how many times does the word "fail" appears in the file? (you can check your result by applying the *count* method on *words*)

In [8]:
print(D['fail'])
print('check:',words.count('fail'))

19
check: 19


3) In general (i.e. not specifically for this file) what are the running times for a) creating the dictionary and b) looking up a word count in the dictionary?

a) Creating the dictionary requires one key lookup per word and either an insertion  or an addition, both of which are assumed to require constant time, so the running time will be $O(N)$ for N words. (key lookup is also constant time)

b) Key lookup and accessing the corresponding value are both constant time, so the total running time is $O(1)$